In [23]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [24]:
root_dir = '../slices'

In [26]:
slice_df = pd.read_csv("../slice_filenames.csv")
slice_df.head()

,Unnamed: 0,fold,filename,classID
0,0,1,46654-6-0-0.wav_0.npy,6
1,1,1,46654-6-0-0.wav_1.npy,6
2,2,1,46654-6-0-0.wav_0.npy,6
3,3,1,46654-6-0-0.wav_1.npy,6
4,4,1,57320-0-0-24.wav_0.npy,0


In [49]:
class SignalDataset(Dataset):
    def __init__(self, slice_df):
        self.slice_df = slice_df

    def __len__(self):
        return len(self.slice_df)

    def __getitem__(self, idx):
        row = self.slice_df.iloc[idx,:]
        filename = row['filename']
        fold = row['fold']
        x = np.load(os.path.join(root_dir, f"fold{fold}", filename))
        return x, row['classID']

In [50]:
train_df = slice_df[(slice_df['fold'] != 8) & (slice_df['fold'] != 9)]
test_df = slice_df[(slice_df['fold'] == 8) | (slice_df['fold'] == 9)]

In [51]:
train_ds = SignalDataset(train_df)
test_ds = SignalDataset(test_df)

In [52]:
train_dl = DataLoader(train_ds)
test_dl = DataLoader(test_ds)

In [65]:
def train(dataloader, model, loss_fn, optimizer, update_params=True, print_loss=False):
    for (X, y) in dataloader:
        # Compute prediction and loss
        print(X.shape)
        model.train()
        pred = model(X[:,:-1], X[:,-1].int())
        loss = loss_fn(pred, y)

        if(update_params):
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if(print_loss):
            print(loss.item())

In [54]:
def avg_loss(model, loss_fn, dataloader):
    num_batches = len(dataloader)
    loss = 0
    with torch.no_grad():
        for (X, y) in dataloader:
            pred = model(X[:,:-1], X[:,-1].int())
            loss += loss_fn(pred, y).item()
    loss /= num_batches
    return loss

def train_loop(train_dataloader, val_dataloader, model, loss_fn, optimizer, n_epochs=10):
    train_losses, val_losses = [], []
    for _ in range(n_epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        train_loss = avg_loss(model, loss_fn, train_dataloader)
        val_loss = avg_loss(model, loss_fn, val_dataloader)
        train_losses += [train_loss]
        val_losses += [val_loss]
        print(f"train_loss: {train_loss}, val_loss: {val_loss}")
    return train_losses, val_losses

In [55]:
class SignalModel(nn.Module):
    def __init__(self):
        super(SignalModel, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv1d(1, 16, 3),
            nn.Conv1d(16, 32, 3),
            nn.Conv1d(32, 64, 3),
        )

    def forward(self, x):
        return self.layer(x)

In [56]:
loss_fn = nn.BCEWithLogitsLoss()
model = SignalModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [64]:
print("test")
print("test")

test
test


In [66]:
print("test")
train(train_dl, model, loss_fn, optimizer)

test
torch.Size([1, 44100])


TypeError: forward() takes 2 positional arguments but 3 were given